这是 **工程实践与科技创新Ⅳ-J (CS3507)** 的第二次课程作业，请仔细阅读以下注意事项：

1. 注意作业截止日期，提交作业只需上传 `ipynb` 文件即可，<font color=red>请务必保留所有单元格的运行结果！</font>
2. 本次提供了少量参考代码，大部分可直接使用也可自行修改，不能更改或需要更改处会标注
3. 本次作业分三部分，三部分得分占比约50%，25%，25% ，请留意2、3部分有一定的问答
4. 建议本地补全并运行，保留自己相应的结果

在本次作业中，我们将实现与Attention相关的几个模块。

Task1: 实现自注意力模块的前向过程与反向过程，最终输出给定输入相应的output, d_W_q, d_W_k, d_W_v

一些提示：对于注意力机制还不太熟悉的同学可以回顾原文https://arxiv.org/abs/1706.03762
或是一些优秀专栏https://www.zhihu.com/tardis/bd/art/414084879?source_id=1001 加深对其的了解。
关键其实就是实现相应公式，再反向求梯度。
<center><img src="attention.png"/></center>

反向求梯度推导过程：
$$
\begin{align*}
& 设输入矩阵为X. \\
& \left\{\begin{array}{l} 
Q = X\cdot W_Q, K = X\cdot W_K, V=X\cdot W_V \\
Score = Q\cdot K^T \\
Temp = \frac{Score}{\sqrt{d_K}} \\
W_{Score} = softmax(Temp) \\
Context = W_{Score}\cdot V 
\end{array}\right.\\


& 由求梯度公式：&\\
& Y = X\cdot W \Rightarrow 
\left\{\begin{array}{l} 
dW = X^T\cdot dY \\
dX = dY\cdot W^T 
\end{array}\right.\\

& 可得：\\
& \left\{\begin{array}{l} 
dW_V = X^T\cdot dV \\
dV = W_{Score}^T\cdot dContext
\end{array}\right.
\Rightarrow dW_V = X^T\cdot (W_{Score}^T\cdot dContext) \\

& \left\{\begin{array}{l} 
dW_Q = X^T\cdot dQ \\
dQ = dScore\cdot (K^T)^T = dScore\cdot K \\
dScore = \frac{dTemp}{\sqrt{d}} \\
dTemp = softmax'(Temp) * dW_{Score} \\
dW_{Score} = dContext\cdot V^T 
\end{array}\right.
\Rightarrow dW_Q = X^T\cdot (\frac{softmax'(Temp)}{\sqrt{d}} * (dContext\cdot V^T))\cdot K \\

& \left\{\begin{array}{l} 
dW_K = X^T\cdot dK \\
dK = E\cdot dK^T \\
dK^T = Q^T\cdot dScore \\
dScore = ...
\end{array}\right.
\Rightarrow dW_K = X^T\cdot Q^T\cdot (\frac{softmax'(Temp)}{\sqrt{d}} * (dContext\cdot V^T)) &
\end{align*}
$$

In [2]:
import numpy as np

np.random.seed(42)

# 你应该会需要用到softmax和它的导数，可以直接用下面定义好的，或者自己实现。
def softmax(x, axis=-1):
    exp_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
    softmax_x = exp_x / np.sum(exp_x, axis=axis, keepdims=True)
    
    return softmax_x

def softmax_derivative(x, axis=-1):
    softmax_x = softmax(x, axis)
    softmax_derivative_x = softmax_x * (1 - softmax_x)
    
    return softmax_derivative_x

def self_attention(input_sequence):
    
    # 通过np随机初始化权重矩阵
    W_q = np.random.randn(3, 3)   # embedding_dim,attention_dim 3,3
    W_k = np.random.randn(3, 3)
    W_v = np.random.randn(3, 3)

    ##############################################
    # TODO
    # 实现前向过程与反向过程
    ##############################################

    # 计算Q、K、V
    Q = np.dot(input_sequence, W_q)
    K = np.dot(input_sequence, W_k)
    V = np.dot(input_sequence, W_v)
    
    # 计算注意力分数
    Score = np.dot(Q, K.T)
    
    # 计算注意力权重
    sqrt_d_k = K.shape[1]**0.5
    Temp = Score / sqrt_d_k
    W_score = softmax(Temp)
    
    # 计算上下文向量，即公式结果
    context = np.dot(W_score, V)
    
    # 计算中间过程梯度
    d_context = np.ones_like(context)  # 上下文向量的梯度，可理解为将context中所有元素求和作为结果，对其中每个元素的梯度都为1

    # 计算权重矩阵的梯度
    d_W_q = np.dot(input_sequence.T, np.dot(softmax_derivative(Temp) / sqrt_d_k * np.dot(d_context, V.T), K))
    d_W_k = np.dot(input_sequence.T, np.dot(Q.T, softmax_derivative(Temp) / sqrt_d_k * np.dot(d_context, V.T)))
    d_W_v = np.dot(input_sequence.T, np.dot(W_score.T, d_context))

    ##############################################
    # 结束您的代码
    ##############################################

    return context, d_W_q, d_W_k, d_W_v


# 以下部分无需修改
# 输入
input_sequence = np.array([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9]])  #无实际意义

# 计算自注意力和梯度
output, d_W_q, d_W_k, d_W_v = self_attention(input_sequence)

print("输出:")
print(output)
print("W_q 的梯度:")
print(d_W_q)
print("W_k 的梯度:")
print(d_W_k)
print("W_v 的梯度:")
print(d_W_v)

输出:
[[-0.79341887 -0.45775989 -0.81055327]
 [-0.78585269 -0.452197   -0.80554836]
 [-0.7782951  -0.44664042 -0.80054912]]
W_q 的梯度:
[[-0.02401657  2.01914624  1.01479483]
 [-0.0300841   2.52596572  1.26955251]
 [-0.03615163  3.0327852   1.5243102 ]]
W_k 的梯度:
[[-0.07233604 -0.16692219 -0.25532755]
 [-0.14370815 -0.33175139 -0.50768981]
 [-0.21508025 -0.49658058 -0.76005207]]
W_v 的梯度:
[[1.16983906 1.16983906 1.16983906]
 [1.46983906 1.46983906 1.46983906]
 [1.76983906 1.76983906 1.76983906]]


Task2: 实现正余弦位置编码。
位置编码（Positional Encoding）在前面的文章（ https://arxiv.org/abs/1706.03762 ）中被引入，可以帮助模型能够更好地捕捉到序列中不同位置之间的关系，从而更准确地理解和表示输入序列。
这里我们采用原文中的正余弦位置编码
<center><img src="PE.png"/></center>

完成之后思考并回答，为什么在Attention中开始引入位置编码，为什么最初采用这种正余弦编码的方式。

回答：
+ 在注意力机制中引入位置编码是因为传统的注意力机制（比如基于点积的自注意力机制）缺乏位置信息。在自然语言处理等任务中，单词的顺序对理解文本至关重要。因此，为了使模型能够考虑到单词的位置信息，就需要向输入中引入位置编码。它们以某种方式（比如使用正弦和余弦函数）编码单词在句子中的位置信息。这样，模型就可以根据单词的位置来调整其在注意力机制中的权重，从而更好地理解输入的顺序信息。
+ 正弦和余弦函数具有不同频率和相位，可以编码不同位置的信息。它们具有周期性，能够捕捉到不同位置之间的相对位置信息。这种编码方式可以将位置信息嵌入到模型中，帮助模型在学习中更好地理解不同位置之间的关系，而且相对简单易于计算，能提高模型在处理序列数据时的性能。


In [3]:
def positional_encoding(pos_id_list, d_model):
    """
    pos_id为给定一个序列中的位置标识符，如0，1，2，3，...
    这里给出一包含多个位置标识符的列表pos_id_list，要求对相应位置实现位置编码
    d_model为位置编码维度，一般与使用模型中的隐藏层维度相关
    pos_enc为输出，其中保留了全部的结果
    """
    ##############################################
    # TODO
    # 实现正余弦位置编码
    ##############################################

    n = len(pos_id_list)

    pos_enc = {}

    for pos in pos_id_list:
        pos_enc[pos] = [0] * 6
        for i in range(0, d_model):
            if i % 2 == 0: 
                pos_enc[pos][i] = np.sin(pos / (10000**(i / d_model)))
            else:
                pos_enc[pos][i] = np.cos(pos / (10000**((i - 1) / d_model)))
    ##############################################
    # 结束您的代码
    ##############################################    
    return pos_enc

pos_id_list = [12,1,8,9,3]
d_model = 6
pos_enc = positional_encoding(pos_id_list, d_model)

print(pos_enc)

{12: [-0.5365729180004349, -0.5365729180004349, 0.5286341178588567, 0.5286341178588567, 0.02585033637662907, 0.02585033637662907], 1: [0.8414709848078965, 0.8414709848078965, 0.046399223464731285, 0.046399223464731285, 0.0021544330233656045, 0.0021544330233656045], 8: [0.9893582466233818, 0.9893582466233818, 0.3628524110177816, 0.3628524110177816, 0.017234624199596284, 0.017234624199596284], 9: [0.4121184852417566, 0.4121184852417566, 0.40569856994848597, 0.40569856994848597, 0.019388697233126855, 0.019388697233126855], 3: [0.1411200080598672, 0.1411200080598672, 0.13879810108005056, 0.13879810108005056, 0.006463259070189646, 0.006463259070189646]}


Task3：调用transformers库，实现对给定文本的逐词编码（embedding）。
其中涉及词嵌入（Word Embedding）技术，其将离散的词语表示转换为连续的向量表示。
给定文本：

text1="This book is very interesting, I recommend it to you."

text2="Having a cup of coffee in the morning can be refreshing."

text3="He is learning programming and making rapid progress."


你需要：1）将文本转为一个word的list；2）调用transformers库的AutoTokenizer和AutoModel等加载模型实现逐词编码

除代码实现外，请简要说明所使用的词嵌入模型的基本原理。可探索对于中文句子的编码方式（会需要用到jieba等库实现中文分词）

以下为用gensim库的一个简单示例说明流程。最终你需要给出你实现的代码以及结果并做必要的说明和分析。

回答：
+ BERT（Bidirectional Encoder Representations from Transformers）模型使用了两种嵌入技术：词嵌入（Word Embeddings）和位置嵌入（Position Embeddings）。这些嵌入技术共同构成了BERT模型的输入表示。
    1. 词嵌入（Word Embeddings）：BERT使用了基于上下文的词嵌入技术。在预训练阶段，BERT模型首先使用WordPiece或者Byte Pair Encoding（BPE）等子词分词方法将单词划分成子词单元。然后，每个子词单元都被映射到一个向量空间中的固定维度的词嵌入向量。这些词嵌入向量是模型学习到的，能够捕捉单词的语义信息。
    2. 位置嵌入（Position Embeddings）：由于BERT模型是基于Transformer结构的，它不像循环神经网络（RNN）那样具有自然的位置信息。因此，为了使模型能够理解输入序列的顺序信息，BERT模型引入了位置嵌入。位置嵌入是一种表示输入序列中每个位置的向量，它们通过正弦和余弦函数的周期性编码来表征不同位置的信息。这种编码方式能够捕捉到位置之间的相对关系，并为模型提供必要的位置信息。
    
    通过结合词嵌入和位置嵌入，BERT模型能够在预训练阶段学习到更丰富和更有意义的输入表示，从而在各种自然语言处理任务中取得较好的性能。

+ 我实现的代码使用的是bert-base-uncased模型，是 Google 在 2018 年发布的 BERT 模型的一个版本，它是基于 Transformer 结构构建的双向编码器（Bidirectional Encoder）。因为无法连接huggingface，所以将模型下载到了本地。首先调用分词器对句子进行分词，然后将分词转化为词汇表索引。最后对每个词调用模型的输入词嵌入层，得到每个词的词嵌入向量。

In [3]:
##############################################
# 运行自己代码时注意安装相应库，可通过
# !pip install jieba
# 这样的形式在jupyter中安装
# 模型下载如遇到huggingface连不上之类的问题，可手动下到本地或通过https://hf-mirror.com/等方法解决
##############################################
import gensim.downloader
import jieba

# 下载并加载预训练模型（遇到需要翻墙的问题可以自己去下载模型到本地）
model = gensim.downloader.load("word2vec-google-news-300")
# 对于无需分词的单个单词，可直接获得"abandon"的嵌入向量
embedding = model['abandon']
print(embedding)
# 给定了文本句子
text = "我爱中国"
# 需要分词处理，得到一个list
# 中文可能需要调用jieba等库
tokens = jieba.lcut(text)
print(tokens)
# tokens = text.split()  # 英文文本分词为单词列表
embeddings = [model[word] for word in tokens]  # 获取每个单词的嵌入向量
print(embeddings)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\11910\AppData\Local\Temp\jieba.cache


[ 7.65991211e-03  7.86132812e-02  1.09375000e-01  3.39843750e-01
 -2.08984375e-01  4.46777344e-02 -3.66210938e-02 -4.19921875e-02
  1.92382812e-01  1.39648438e-01 -1.54296875e-01 -3.02734375e-02
 -1.13769531e-01  2.16796875e-01 -1.40625000e-01  3.83300781e-02
  1.39648438e-01 -1.16210938e-01 -8.05664062e-02 -1.04003906e-01
  7.08007812e-02  1.52587891e-02  1.06933594e-01  2.71484375e-01
  1.93359375e-01  1.42578125e-01 -1.48315430e-02  1.94335938e-01
  1.66992188e-01 -3.08593750e-01  3.06640625e-01  1.64062500e-01
 -3.57421875e-01 -5.70678711e-03 -1.37695312e-01  2.03857422e-02
  1.31835938e-01  8.74023438e-02  7.47070312e-02 -3.27148438e-02
  3.20312500e-01  9.71679688e-02  1.72851562e-01 -6.73828125e-02
 -1.81640625e-01  5.24902344e-03 -1.59179688e-01 -1.74560547e-02
  6.73828125e-02  1.15722656e-01 -9.15527344e-03  1.99218750e-01
  5.81054688e-02 -1.87500000e-01 -3.83300781e-02 -1.50390625e-01
 -1.98242188e-01 -3.53515625e-01 -4.90722656e-02 -1.47460938e-01
  6.39648438e-02  1.34887

Loading model cost 0.653 seconds.
Prefix dict has been built successfully.


['我', '爱', '中国']
[array([ 0.07910156, -0.28125   ,  0.06640625,  0.03759766, -0.02856445,
        0.18457031,  0.09082031,  0.01098633, -0.04980469,  0.3828125 ,
       -0.23730469,  0.10253906, -0.44726562,  0.26367188, -0.17480469,
       -0.0168457 ,  0.05786133,  0.15527344, -0.28515625,  0.00872803,
        0.08886719, -0.16113281,  0.49609375,  0.03222656, -0.22070312,
       -0.40234375,  0.21386719,  0.09716797, -0.26953125, -0.11328125,
       -0.19335938, -0.01843262,  0.14355469, -0.25390625, -0.20898438,
       -0.19628906,  0.10058594,  0.171875  ,  0.08154297,  0.03063965,
       -0.00318909, -0.20605469, -0.02075195,  0.02087402, -0.09375   ,
       -0.24023438, -0.38085938, -0.22949219, -0.00386047,  0.03613281,
       -0.45117188,  0.40429688,  0.25976562,  0.04150391, -0.11035156,
        0.37109375, -0.265625  , -0.26367188,  0.24121094, -0.09082031,
       -0.15332031,  0.44726562,  0.04736328,  0.22070312,  0.11425781,
       -0.05029297,  0.06835938, -0.08984375, 

In [17]:
# 你可以在本地测试，或将你的代码写到这个单元格内，最终要有相应的结果

import torch
from transformers import AutoTokenizer, AutoModel
# from transformers import BertTokenizer

tokenizer = AutoTokenizer.from_pretrained("./bert-base-uncased") # 加载已下载到本地的模型（放在作业文件同一目录下）
model = AutoModel.from_pretrained("./bert-base-uncased")

text1="This book is very interesting, I recommend it to you."
text2="Having a cup of coffee in the morning can be refreshing."
text3="He is learning programming and making rapid progress."

texts = [text1, text2, text3]
# print(texts)

tokens = [tokenizer.tokenize(text) for text in texts] # 分词器进行分词
print(tokens) # 打印结果

words_id = [tokenizer.convert_tokens_to_ids(token) for token in tokens] # 转化为词汇表索引
print(words_id)

embed = model.get_input_embeddings() # 获取模型的输入词嵌入层

embeddings = [embed(torch.tensor([word_id])) for word_id in words_id] # 对每个词汇表索引（即每个词）调用模型的输入词嵌入层，得到每个词的词嵌入向量。

print(embeddings)

[['this', 'book', 'is', 'very', 'interesting', ',', 'i', 'recommend', 'it', 'to', 'you', '.'], ['having', 'a', 'cup', 'of', 'coffee', 'in', 'the', 'morning', 'can', 'be', 'refreshing', '.'], ['he', 'is', 'learning', 'programming', 'and', 'making', 'rapid', 'progress', '.']]
[[2023, 2338, 2003, 2200, 5875, 1010, 1045, 16755, 2009, 2000, 2017, 1012], [2383, 1037, 2452, 1997, 4157, 1999, 1996, 2851, 2064, 2022, 27150, 1012], [2002, 2003, 4083, 4730, 1998, 2437, 5915, 5082, 1012]]
[tensor([[[-5.7095e-02,  1.5283e-02, -4.6868e-03,  ..., -3.2484e-03,
           9.7317e-05,  9.4175e-03],
         [ 3.5072e-03, -2.3005e-02,  3.8487e-02,  ..., -3.4511e-02,
          -4.2856e-02, -9.5698e-03],
         [-3.6044e-02, -2.4606e-02, -2.5735e-02,  ...,  3.3691e-03,
          -1.8300e-03,  2.6855e-02],
         ...,
         [ 1.3139e-02,  8.1785e-03, -8.7239e-03,  ...,  1.5858e-02,
          -7.8034e-03,  1.8179e-02],
         [-3.6435e-02,  4.5340e-03,  3.0638e-02,  ...,  1.8992e-02,
           1.13